In [72]:
from bddl.knowledge_base import *

In [67]:
tasks = Task.view_challenge()
for task in tasks:
    # print(task.name, [s.name for s in task.matched_scenes])
    if not task.matched_scenes:
        print(task.name)
    if task.uses_cloth:
        print(task.name, "uses cloth", {x for x in task.synsets if "cloth" in x.property_names})
print(len(tasks), "tasks")

100 tasks


In [82]:
# Starting pick
ig_scenes = {x for x in Scene.all_objects() if x.name.endswith("_int")}
starting_set = {Scene.get(x) for x in {   # ig_scenes | {
    "house_single_floor",
    "house_double_floor_lower",
    "Beechwood_0_garden",
    # "Rs_garden",
}}

In [83]:
tasks_within_capability = set(task for task in Task.all_objects() if not task.uses_cloth)
tasks_that_are_not_selected = tasks_within_capability - set(tasks)
tasks_that_can_be_done_in_scenes = set(task for task in tasks_that_are_not_selected if set(task.matched_scenes) & starting_set)
print("Tasks that are not selected:", len(tasks_that_can_be_done_in_scenes))

Tasks that are not selected: 471


In [84]:
# Toggled on
toggle_tasks = {task for task in tasks_that_can_be_done_in_scenes if any("toggle" in p.name.lower() for p in task.uses_predicates)}
toggle_tasks

{Task(name='changing_light_bulbs-0'),
 Task(name='clean_the_bottom_of_an_iron-0'),
 Task(name='de_ice_a_car-0'),
 Task(name='installing_a_fax_machine-0'),
 Task(name='installing_a_printer-0'),
 Task(name='installing_a_scanner-0'),
 Task(name='prepare_and_cook_prawns-0'),
 Task(name='putting_roast_in_oven-0')}

In [87]:
import random

TARGETS = {Scene.get(k): v for k, v in {
    "house_single_floor": 50,  # encourage hsf
    "house_double_floor_lower": 40,  # encourage lower
    "Beechwood_0_garden": -5000,  # discourage beechwood
    "Rs_garden": -10000,  # discourage beechwood
}.items()}
def assign_activities_balanced(scene_set, dont_assign=None):
    # 2. Initialize assignment and location loads
    assignment = {}
    location_load = {scene: 0 for scene in scene_set}
    location_load[None] = 0  # For unassigned tasks

    # Add a negative count to house single floor to increase its count
    for sc, offset in TARGETS.items():
        if sc not in location_load:
            continue
        location_load[sc] -= offset

    # 3. Shuffle activities to avoid bias from input order (optional but recommended)
    task_set = list(tasks)
    random.shuffle(task_set)

    # Keep trying to assign tasks to the lowest loaded scene
    # until all tasks are assigned or no more options are available
    # while True:
    #     # 1. Check if all tasks are assigned
    #     if all(task in assignment for task in task_set):
    #         break

    #     # 2. Check if there are any unassigned tasks that can be assigned to the current scene set
    #     unassigned_tasks = [task for task in task_set if task not in assignment]
    #     if not unassigned_tasks:
    #         break

    #     # 3. Check if there are any scenes with available capacity
    #     worst_scene = min(location_load, key=location_load.get)
    #     compatible_tasks = [task for task in unassigned_tasks if worst_scene in task.matched_scenes]
    #     if not compatible_tasks:
    #         break

    #     # 4. Randomly select a task from the compatible tasks
    #     task = random.choice(compatible_tasks)

    #     # 5. Assign the task to the scene and update the load
    #     assignment[task] = worst_scene
    #     location_load[worst_scene] += 1

    # 4. Greedy assignment process
    for task in task_set:
        if task in assignment:
            continue

        possible_locations = set(task.matched_scenes) & scene_set

        if dont_assign and task in dont_assign:
            possible_locations = possible_locations - {dont_assign[task]}
        
        possible_locations = list(possible_locations)

        if not possible_locations:
            assignment[task] = None
            location_load[None] += 1
            continue

        # Find minimum load and randomly choose among ties
        min_load = min(location_load[loc] for loc in possible_locations)
        best_options = [loc for loc in possible_locations if location_load[loc] == min_load]
        chosen_location = random.choice(best_options)

        # Assign and update load
        assignment[task] = chosen_location
        location_load[chosen_location] += 1

    # for _ in range(20): # Try to balance the load a bit more
    #     # Go through the tasks and check if any task assigned to a scene that has too many tasks
    #     # can be switched to a scene that has too few tasks.
    #     # This is a greedy approach and may not be optimal, but it should help balance the load.
    #     changed = False

    #     worst_scene = max(location_load, key=location_load.get)
    #     for task, assigned_scene in assignment.items():
    #         if assigned_scene != worst_scene:
    #             continue

    #         # Find a scene with less load to swap with
    #         for other_scene in sorted(scene_set, key=location_load.get):
    #             if other_scene == assigned_scene:
    #                 continue

    #             # Check if the task can be moved to the other scene
    #             if other_scene in task.matched_scenes:
    #                 # Swap the assignment
    #                 assignment[task] = other_scene
    #                 location_load[assigned_scene] -= 1
    #                 location_load[other_scene] += 1
    #                 changed = True
    #                 break

    #         if changed:
    #             break

    #     if not changed:
    #         break

    for sc, offset in TARGETS.items():
        if sc not in location_load:
            continue
        location_load[sc] += offset

    total_load = sum(location_load.values())
    assert total_load == len(tasks), "Total load mismatch: expected {}, got {}".format(len(tasks), total_load)
    return assignment, location_load

In [92]:
# What activities do these scenes cover?
def cover(scene_set):
    return {task for task in tasks if set(task.matched_scenes) & scene_set}

print(len(cover(starting_set)))
assignment, counts = assign_activities_balanced(starting_set)
print(counts)

100
{Scene(name='Beechwood_0_garden'): 11, Scene(name='house_single_floor'): 51, Scene(name='house_double_floor_lower'): 38, None: 0}


In [95]:
# Print the assignments
task_list = """
putting_away_toys-0
can_meat-0

assembling_gift_baskets-0
picking_up_trash-0
putting_away_Halloween_decorations-0
can_vegetables-0
bringing_water-0
cleaning_up_plates_and_food-0
picking_up_toys-0
clean_up_broken_glass-0
carrying_in_groceries-0
collecting_aluminum_cans-0
clearing_the_table_after_dinner-0
loading_the_dishwasher-0
setting_mousetraps-0


hiding_Easter_eggs-0
sorting_mail-0
make_dessert_watermelons-0
rearranging_furniture-0
rearranging_kitchen_furniture-0
putting_up_Christmas_decorations_inside-0
bringing_in_mail-0
bringing_in_wood-0
dispose_of_a_pizza_box-0
opening_windows-0
clean_up_your_desk-0
preserving_meat-0
bringing_glass_to_recycling-0

watering_outdoor_flowers-0


fill_a_bucket_in_a_small_sink-0
can_syrup-0
can_beans-0
carrying_water-0
filling_salt-0
mixing_drinks-0

adding_chemicals_to_pool-0
spraying_for_bugs-0
painting_porch-0
installing_a_fence-0
ice_cookies-0
adding_chemicals_to_lawn-0
watering_indoor_flowers-0
cleaning_bathtub-0


putting_up_shelves-0
putting_on_license_plates-0
installing_smoke_detectors-0
hanging_pictures-0
attach_a_camera_to_a_tripod-0
setup_a_trampoline-0
cook_hot_dogs-0
make_a_steak-0
toast_buns-0
grill_burgers-0
cook_bacon-0
cook_a_brisket-0
cool_cakes-0
setting_the_fire-0
freeze_pies-0
freeze_fruit-0
freeze_vegetables-0
freeze_meat-0
thawing_frozen_food-0
thaw_frozen_fish-0
heating_food_up-0
reheat_frozen_or_chilled_food-0
turning_on_radio-0
mowing_the_lawn-0
installing_a_modem-0
installing_alarms-0
turning_out_all_lights_before_sleep-0



chop_an_onion-0
slicing_vegetables-0
chopping_wood-0
make_spa_water-0
cook_eggplant-0
melt_white_chocolate-0
cook_corn-0
cook_a_crab-0
cook_a_pumpkin-0
boil_water-0
boil_water_in_the_microwave-0
toast_sunflower_seeds-0
toast_coconut-0
cook_chickpeas-0
make_microwave_popcorn-0
cook_soup-0
make_a_strawberry_slushie-0
make_a_milkshake-0
make_chocolate_milk-0

make_cake_mix-0
make_cinnamon_sugar-0
make_a_basic_brine-0
make_a_cappuccino-0
make_biscuits-0
make_chocolate_biscuits-0
make_cookies-0
make_dinner_rolls-0

clean_a_hamper-0
clean_a_tie-0
wash_a_baseball_cap-0


""".strip().split("\n")

In [96]:
for task_name in task_list:
    task = Task.get(task_name)
    if task in assignment:
        print(assignment[task].name)
    else:
        print("")

house_single_floor
house_single_floor

house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_single_floor
house_single_floor
house_double_floor_lower
house_single_floor
house_single_floor
house_single_floor
Beechwood_0_garden
house_single_floor


house_double_floor_lower
house_double_floor_lower
house_single_floor
house_single_floor
house_double_floor_lower
house_single_floor
house_double_floor_lower
house_double_floor_lower
house_single_floor
Beechwood_0_garden
Beechwood_0_garden
house_single_floor
house_double_floor_lower

house_single_floor


house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_single_floor
house_double_floor_lower
house_single_floor

house_single_floor
house_single_floor
house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_double_floor_lower
house_single_floor


house_double_floor_lower
house_single_floor
hous

In [97]:
# Do a second round to assign a second scene to each task. Don't use the first scene assigned to each task.
assignment2, counts2 = assign_activities_balanced(starting_set, dont_assign=assignment)
print(counts2)

{Scene(name='Beechwood_0_garden'): 36, Scene(name='house_single_floor'): 36, Scene(name='house_double_floor_lower'): 5, None: 23}


In [98]:
for task_name in task_list:
    task = Task.get(task_name)
    if assignment2.get(task, None):
        print(assignment2[task].name)
    else:
        print("")

Beechwood_0_garden
Beechwood_0_garden

house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor


house_single_floor
Beechwood_0_garden

Beechwood_0_garden




house_single_floor
house_single_floor
Beechwood_0_garden

house_single_floor
Beechwood_0_garden
house_single_floor
house_single_floor
Beechwood_0_garden


Beechwood_0_garden
house_single_floor

Beechwood_0_garden


house_single_floor
house_single_floor
house_single_floor

house_single_floor
Beechwood_0_garden


house_double_floor_lower
house_single_floor
house_single_floor
house_single_floor
house_single_floor
house_single_floor



house_single_floor
house_double_floor_lower



house_single_floor
Beechwood_0_garden
house_single_floor
house_single_floor
house_single_floor
Beechwood_0_garden
Beechwood_0_garden
house_single_floor

Beechwood_0_garden
Beechwood_0_garden
Beechwood_0_garden
Beechwood_0_garden
Beechwood_0_garden



house_single_floor
house_single_floor

Beechwood_0_garden




house_

In [ ]:
# Greedy algorithm step
def next_pick(scene_set):
    candidates = set(Scene.all_objects()) - scene_set
    current_cover = cover(scene_set)
    advantages = {x: cover(scene_set | {x}) - current_cover for x in candidates}
    return dict(sorted(advantages.items(), key=lambda x: -len(x[1])))
{k: len(v) for k, v in next_pick(starting_set).items()}

In [ ]:
# Run the greedy algorithm
max_cover = len(cover(scenes))
greedy_selection = set(starting_set)
while len(cover(greedy_selection)) < max_cover:
    pick = next_pick(greedy_selection)
    print("Adding", pick)
    greedy_selection.add(pick[0])
print("Final set:", sorted(greedy_selection))
print(len(greedy_selection))

In [ ]:
import itertools, tqdm

print("Max possible cover:", max_cover)

# Run a non-greedy version
def combinatorial_search():
    max_extra_scenes_needed = len(greedy_selection) - len(starting_set)
    print("We will process up to", max_extra_scenes_needed, "extra scenes")
    extra_scene_candidates = scenes - starting_set
    print("We have", len(extra_scene_candidates), "candidates")
    
    def combinatorial_search_n(extra_to_use):
        best_cover = 0
        best_cover_set = set()
        
        print("Processing", extra_to_use, "element combinations")
        for combination_extra in tqdm.tqdm(itertools.combinations(extra_scene_candidates, extra_to_use)):
            combination = starting_set | set(combination_extra)
            combination_cover = len(cover(combination))

            if combination_cover >= best_cover:
                best_cover = combination_cover
                best_cover_set = combination
            
            if combination_cover == max_cover:
                break
                
        return best_cover_set, best_cover
    
    for extra_to_use in range(1, max_extra_scenes_needed + 1):
        combination, combination_cover = combinatorial_search_n(extra_to_use)
        print(f"Best total with {extra_to_use} elements: {combination_cover}. Scenes: {sorted(combination - starting_set)}")
        
        if combination_cover == max_cover:
            print("Final set.")
            return
            
combinatorial_search()

In [ ]:
# Pomaria-advantage
print(len(cover(starting_set | {"Pomaria_0_garden"})) - len(cover(starting_set)))

In [ ]:
final_set = starting_set | {'Pomaria_0_garden', 'grocery_store_cafe', 'office_large', 'restaurant_brunch'}
len(cover(final_set))

In [ ]:
print(final_set - ig_scenes)